# asyncio

<https://zhuanlan.zhihu.com/p/25228075>

asyncio是python 3.4中新增的模块，它提供了一种机制，使得你可以用协程（coroutines）、IO复用（multiplexing I/O）在单线程环境中编写并发模型。

根据官方说明，asyncio模块主要包括了：

- 具有特定系统实现的事件循环（event loop）;

- 数据通讯和协议抽象（类似Twisted中的部分);

- TCP，UDP,SSL，子进程管道，延迟调用和其他;

- Future类;

- yield from的支持;

- 同步的支持;

- 提供向线程池转移作业的接口;

下面来看下asyncio的一个例子：

注意：不要再jupyter里运行，最好用pycharm☺


## 示例一

```py
import asyncio

async def compute(x, y):
    print("Compute %s + %s ..." % (x, y))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    result = await compute(x, y)
    print("%s + %s = %s" % (x, y, result))

loop = asyncio.get_event_loop()
loop.run_until_complete(print_sum(1, 2))
loop.close()
```

![](../assets/yield.png)

当事件循环开始运行时，它会在Task中寻找coroutine来执行调度，因为事件循环注册了print_sum()，因此print_sum()被调用，执行result = await compute(x, y)这条语句（等同于result = yield from compute(x, y)），因为compute()自身就是一个coroutine，因此print_sum()这个协程就会暂时被挂起，compute()被加入到事件循环中，程序流执行compute()中的print语句，打印”Compute %s + %s …”，然后执行了await asyncio.sleep(1.0)，因为asyncio.sleep()也是一个coroutine，接着compute()就会被挂起，等待计时器读秒，在这1秒的过程中，事件循环会在队列中查询可以被调度的coroutine，而因为此前print_sum()与compute()都被挂起了，因此事件循环会停下来等待协程的调度，当计时器读秒结束后，程序流便会返回到compute()中执行return语句，结果会返回到print_sum()中的result中，最后打印result，事件队列中没有可以调度的任务了，此时loop.close()把事件队列关闭，程序结束。

## 示例二

```py
# -*- coding: utf-8 -*-
import asyncio
from datetime import datetime
# import nest_asyncio
# nest_asyncio.apply()


async def add(a, b):
    await asyncio.sleep(1)
    return a + b


async def master_thread(loop):
    print("{} master: 1+2={}".format(datetime.now(), await add(1, 2)))


def slave_thread(loop):
    # 注意：这不是 coroutine 函数
    import time
    time.sleep(2)

    f = asyncio.run_coroutine_threadsafe(add(1, 2), loop)
    print("{} slave: 1+2={}".format(datetime.now(), f.result()))


async def main(loop):
    await asyncio.gather(
        master_thread(loop),
        # 线程池内执行
        loop.run_in_executor(None, slave_thread, loop),
    )


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main(loop))
    loop.close()
    
```

## 示例三

下面代码中模拟了使用 2 个线程管理 event loop 的情况,

其中子线程（thread 函数）设定并启用了一个 999 次的 task 任务。

而在主线程中，添加了一个 5 次的 task 任务。我设置了一个 While 循环来检查这项任务是否已经运行完毕，如若完毕则打印出 main done: 5。

子线程独立于主线程，直到运行完才退出。


```py
import asyncio
import threading

# 模拟协程
async def task(c, i):
    for _ in range(i):
        print(c)
        await asyncio.sleep(1)
    return i

def thread(loop):  # 异步程序
    asyncio.set_event_loop(loop)
    asyncio.ensure_future(task('sub thread', 999))
    loop.run_forever()

def main():
    threading.Thread(target=thread, args=(asyncio.get_event_loop(), )).start()

    # 同步代码开始
    future = asyncio.ensure_future(task('main thread', 5))
    while not future.done():
        time.sleep(1)
    print('main done: %s' % future.result())


if __name__ == '__main__':
    main()
```

## 示例三：豆瓣爬虫

```py
# -*- coding:utf-8 -*-
from lxml import etree
from time import time
import asyncio
import aiohttp

url = 'https://movie.douban.com/top250'


async def fetch_content(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.text()


async def parse(url):
    page = await fetch_content(url)
    html = etree.HTML(page)

    xpath_movie = '//*[@id="content"]/div/div[1]/ol/li'
    xpath_title = './/span[@class="title"]'
    xpath_pages = '//*[@id="content"]/div/div[1]/div[2]/a'

    pages = html.xpath(xpath_pages)
    fetch_list = []
    result = []
    # 第一页电影条目
    for element_movie in html.xpath(xpath_movie):
        result.append(element_movie)
    # 获取分页地址url
    for p in pages:
        fetch_list.append(url + p.get('href'))
    # 协程队列
    tasks = [fetch_content(url) for url in fetch_list]
    # 等待全部协程完成，获得每个协程的返回值（分页的html源码），列表顺序与传入顺序相同
    pages = await asyncio.gather(*tasks)

    for page in pages:
        html = etree.HTML(page)
        for element_movie in html.xpath(xpath_movie):
            result.append(element_movie)
    for i, movie in enumerate(result, 1):
        # 输出： 序号 电影名
        title = movie.find(xpath_title).text
        # print(i, title)


def main():
    loop = asyncio.get_event_loop()
    start = time()
    # 重复请求5次（串行），模拟多任务
    for i in range(5):
        loop.run_until_complete(parse(url))
    end = time()
    print('Cost {} seconds'.format((end - start) / 5))
    loop.close()


if __name__ == '__main__':
    main()

```